In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import io
import os
import fileinput

In [2]:
data_dir = '/Users/djemec/data/'
code_dir = '/Users/djemec/code/variant_effect_predict_ml/'

geno_100 = '1k_genome/chr1-part-1_tal.vcf.gz'
annot_path = 'dbsnp/dbsnp.chr1.vcf.gz'

## TAL 1 Gene
https://www.genecards.org/cgi-bin/carddisp.pl?gene=TAL1
TAL BHLH Transcription Factor 1, Erythroid Differentiation Factor
Enables several functions, including DNA-binding transcription factor activity; E-box binding activity; and histone deacetylase binding activity. Involved in several processes, including myeloid cell differentiation; positive regulation of cellular component organization; and positive regulation of erythrocyte differentiation. Located in chromatin and nucleoplasm. Part of transcription regulator complex. Implicated in acute lymphoblastic leukemia [provided by Alliance of Genome Resources, Nov 2021]

Latest Assembly
chr1:47,216,290-47,232,389(GRCh38/hg38)
Size:16,100 bases
Orientation:Minus strand

In [3]:
tal1_pos = 'chr1:47216290-47232389'
geno_100_tal = data_dir + '1k_genome/synth_1k_tal.vcf'
geno_head_tal = data_dir + '1k_genome/synth_1k_tal_h.vcf'
geno_headc_tal = data_dir + '1k_genome/synth_1k_tal_hc.vcf'
header = data_dir + '1k_genome/header.txt'
annot_100_output = data_dir + '1k_genome/synth_1k_tal.annot.vcf'
annot_100_clean_output = data_dir + '1k_genome/synth_1k_tal_clean.annot.vcf'
chr_remap = 'dbsnp/chr_remap.txt'

In [4]:
#!bcftools view -r {tal1_pos} {data_dir + geno_100} > {geno_100_tal}

In [5]:
# !bcftools view -h {geno_100_tal} > {header}
# sb = '''##FORMAT=<ID=SB,Number=4,Type=Integer,Description="Per-sample component statistics which comprise the Fisher's Exact Test to detect strand bias.">'''
# rnc = '''##FORMAT=<ID=RNC,Number=1,Type=Integer,Description="SOMETHING">'''
# for line in fileinput.FileInput(header,inplace=1):
#     add=True
#     if '##FORMAT=<ID=PL' in line and add:
#         line=line.replace(line,line+sb+'\n'+rnc+'\n')
#     print(line.strip('\n'))
# !bcftools reheader -h {header} {geno_100_tal} > {geno_head_tal}
# !bcftools annotate --rename-chrs {data_dir + chr_remap} {geno_head_tal} -o {geno_headc_tal}
# !bgzip {geno_headc_tal}
# !tabix {geno_headc_tal + '.gz'}

In [6]:
# !bcftools annotate -a {data_dir + annot_path} -c ID,INFO -o {annot_100_output} {geno_headc_tal + '.gz'}

In [7]:
# Create blank vcf with  only GT, NSF, NSN, ASS, DSS 
#! bcftools annotate -x ^FORMAT/GT,^INFO/NSF,^INFO/NSN,^INFO/ASS,^INFO/DSS,^INFO/VC -o {annot_100_clean_output} {annot_100_output}

# annotate into blank file 

In [8]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

In [9]:
df_a = read_vcf(annot_100_clean_output)
df_a[['VC','impact']] = df_a.INFO.str.split(';',expand=True)
df_a.info()
df_a.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Columns: 1011 entries, CHROM to impact
dtypes: int64(1), object(1010)
memory usage: 3.0+ MB


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_1_0,...,sample_1_992,sample_1_993,sample_1_994,sample_1_995,sample_1_996,sample_1_997,sample_1_998,sample_1_999,VC,impact
0,1,47219343,rs764749756,A,G,7678.43,PASS,VC=SNV,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=SNV,None
1,1,47219348,rs1385978177,T,C,213.77,RF,VC=SNV,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=SNV,None
2,1,47219350,rs1028235803,G,A,1639.69,PASS,VC=SNV,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=SNV,None
3,1,47219370,rs1316319021,A,T,585.18,PASS,VC=SNV,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=SNV,None
4,1,47219382,rs1361035105,C,A,172.71,PASS,VC=SNV,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=SNV,None


In [10]:
df_a.dropna(subset=['impact'], inplace=True)
df_a.info()
df_a

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 123 to 368
Columns: 1011 entries, CHROM to impact
dtypes: int64(1), object(1010)
memory usage: 71.2+ KB


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_1_0,...,sample_1_992,sample_1_993,sample_1_994,sample_1_995,sample_1_996,sample_1_997,sample_1_998,sample_1_999,VC,impact
123,1,47219854,rs1428072561,T,TG,896.55,PASS,VC=DIV;NSF,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=DIV,NSF
143,1,47219902,rs1267529115,CT,C,818.6,RF,VC=DIV;NSF,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=DIV,NSF
154,1,47219920,rs765140893,CACCAG,"C,TACCAG",83098.1,RF;AC0,VC=DIV;NSF,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=DIV,NSF
157,1,47219923,rs1364999495,CAG,C,102973,RF,VC=DIV;NSF,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=DIV,NSF
165,1,47219935,rs1222733270,CAGGGT,C,31.26,RF;AC0,VC=DIV;NSF,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=DIV,NSF
307,1,47225703,rs1289673879,GC,"G,GCC,TC",6194.81,RF,VC=DIV;NSF,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=DIV,NSF
348,1,47225830,rs758403681,TC,T,2215.89,PASS,VC=DIV;NSF,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=DIV,NSF
353,1,47225846,rs1228143364,G,A,221.08,PASS,VC=SNV;NSN,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=SNV,NSN
368,1,47225890,rs1161842453,C,A,34.25,RF;AC0,VC=SNV;ASS,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,VC=SNV,ASS


In [11]:
samp_df = df_a[[s for s in df_a.columns if s.startswith('s')]]
samp_df = samp_df.T
samp_df.head()

,123,143,154,157,165,307,348,353,368
sample_1_0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
sample_1_1,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
sample_1_2,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
sample_1_3,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
sample_1_4,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [12]:
plch = pd.DataFrame(samp_df.agg(lambda x: len('/'.join(x.values))- \
                             '/'.join(x.values).count('/')- \
                             '/'.join(x.values).count('0'),
           axis=1))
plch

,0
sample_1_0,0
sample_1_1,0
sample_1_2,0
sample_1_3,0
sample_1_4,0
...,...
sample_1_995,0
sample_1_996,0
sample_1_997,0
sample_1_998,0


In [13]:
joined_df = samp_df.join(plch)
joined_df = joined_df[[joined_df.columns[-1]]].reset_index()
joined_df.columns= ['sample','impact_ct']

In [14]:
# have sample id and impact count
joined_df.info()
joined_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sample     1000 non-null   object
 1   impact_ct  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


,sample,impact_ct
0,sample_1_0,0
1,sample_1_1,0
2,sample_1_2,0
3,sample_1_3,0
4,sample_1_4,0


In [15]:
df = read_vcf(annot_100_clean_output)
columns = [s for s in df.columns if s.startswith('s')]
columns =  ['CHROM','POS','REF','ALT'] + columns
seq_df = df[columns]
seq_df

,CHROM,POS,REF,ALT,sample_1_0,sample_1_1,sample_1_2,sample_1_3,sample_1_4,sample_1_5,...,sample_1_990,sample_1_991,sample_1_992,sample_1_993,sample_1_994,sample_1_995,sample_1_996,sample_1_997,sample_1_998,sample_1_999
0,1,47219343,A,G,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,1,47219348,T,C,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,1,47219350,G,A,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,1,47219370,A,T,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,1,47219382,C,A,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,1,47225933,C,A,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
386,1,47225934,G,A,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
387,1,47225936,C,"T,CCGGTCAT",0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/.,0/0,0/0,0/0
388,1,47225939,T,TGTGGGCA,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [16]:
seq_df.ALT.str.split(',',expand=True)

,0,1,2,3,4,5
0,G,None,None,None,None,None
1,C,None,None,None,None,None
2,A,None,None,None,None,None
3,T,None,None,None,None,None
4,A,None,None,None,None,None
...,...,...,...,...,...,...
385,A,None,None,None,None,None
386,A,None,None,None,None,None
387,T,CCGGTCAT,None,None,None,None
388,TGTGGGCA,None,None,None,None,None


In [17]:
seq_df[['alt1','alt2','alt3','alt4','alt5','alt6']] = seq_df.ALT.str.split(',',expand=True)
seq_df.info()
seq_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Columns: 1010 entries, CHROM to alt6
dtypes: int64(1), object(1009)
memory usage: 3.0+ MB


/var/folders/5x/w65wy5r52vx0z4vr0yngtdvc0000gn/T/ipykernel_34530/2487710511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df[['alt1','alt2','alt3','alt4','alt5','alt6']] = seq_df.ALT.str.split(',',expand=True)
/var/folders/5x/w65wy5r52vx0z4vr0yngtdvc0000gn/T/ipykernel_34530/2487710511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df[['alt1','alt2','alt3','alt4','alt5','alt6']] = seq_df.ALT.str.split(',',expand=True)
/var/folders/5x/w65wy5r52vx0z4vr0yngtdvc0000gn/T/ipykernel_34530/2487710

,CHROM,POS,REF,ALT,sample_1_0,sample_1_1,sample_1_2,sample_1_3,sample_1_4,sample_1_5,...,sample_1_996,sample_1_997,sample_1_998,sample_1_999,alt1,alt2,alt3,alt4,alt5,alt6
0,1,47219343,A,G,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,G,None,None,None,None,None
1,1,47219348,T,C,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,C,None,None,None,None,None
2,1,47219350,G,A,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,A,None,None,None,None,None
3,1,47219370,A,T,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,T,None,None,None,None,None
4,1,47219382,C,A,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,A,None,None,None,None,None


### Bio Note
for germline genomic data in exons, no-calls are rare and can be assumed to be the ref given the selective pressure in the region

In [18]:
def seq_map(dfs):
    ref = dfs.REF
    alt1 = dfs.loc['alt1']
    alt2 = dfs.loc['alt2']
    alt3 = dfs.loc['alt3']
    alt4 = dfs.loc['alt4']
    alt5 = dfs.loc['alt5']
    alt6 = dfs.loc['alt6']
    for i in [s for s in dfs.index if s.startswith('s')]:
        pre_seq = dfs.loc[i]
        seq_r = []
        for l in pre_seq.split('/'):
            if l == '.':
                seq_r.append('_')
                # for some pop-gen this can also be ref
            elif l == '1':
                seq_r.append(alt1)
            elif l == '2':
                seq_r.append(alt2)
            elif l == '3':
                seq_r.append(alt3)
            elif l == '4':
                seq_r.append(alt4)
            elif l == '5':
                seq_r.append(alt5)
            elif l == '6':
                seq_r.append(alt6)
            else:
                seq_r.append(ref)
        dfs.update({i:seq_r})
    return dfs


In [19]:
seq_int_df = seq_df.apply(seq_map, axis=1)

In [20]:
seq_int_df

,CHROM,POS,REF,ALT,sample_1_0,sample_1_1,sample_1_2,sample_1_3,sample_1_4,sample_1_5,...,sample_1_996,sample_1_997,sample_1_998,sample_1_999,alt1,alt2,alt3,alt4,alt5,alt6
0,1,47219343,A,G,"[A, A]","[A, A]","[A, A]","[A, A]","[A, A]","[A, A]",...,"[A, A]","[A, A]","[A, A]","[A, A]",G,None,None,None,None,None
1,1,47219348,T,C,"[T, T]","[T, T]","[T, T]","[T, T]","[T, T]","[T, T]",...,"[T, T]","[T, T]","[T, T]","[T, T]",C,None,None,None,None,None
2,1,47219350,G,A,"[G, G]","[G, G]","[G, G]","[G, G]","[G, G]","[G, G]",...,"[G, G]","[G, G]","[G, G]","[G, G]",A,None,None,None,None,None
3,1,47219370,A,T,"[A, A]","[A, A]","[A, A]","[A, A]","[A, A]","[A, A]",...,"[A, A]","[A, A]","[A, A]","[A, A]",T,None,None,None,None,None
4,1,47219382,C,A,"[C, C]","[C, C]","[C, C]","[C, C]","[C, C]","[C, C]",...,"[C, C]","[C, C]","[C, C]","[C, C]",A,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,1,47225933,C,A,"[C, C]","[C, C]","[C, C]","[C, C]","[C, C]","[C, C]",...,"[C, C]","[C, C]","[C, C]","[C, C]",A,None,None,None,None,None
386,1,47225934,G,A,"[G, G]","[G, G]","[G, G]","[G, G]","[G, G]","[G, G]",...,"[G, G]","[G, G]","[G, G]","[G, G]",A,None,None,None,None,None
387,1,47225936,C,"T,CCGGTCAT","[C, C]","[C, C]","[C, C]","[C, C]","[C, C]","[C, C]",...,"[C, _]","[C, C]","[C, C]","[C, C]",T,CCGGTCAT,None,None,None,None
388,1,47225939,T,TGTGGGCA,"[T, T]","[T, T]","[T, T]","[T, T]","[T, T]","[T, T]",...,"[T, T]","[T, T]","[T, T]","[T, T]",TGTGGGCA,None,None,None,None,None


In [21]:
seq_int_df = seq_int_df.set_index('POS')
seq_int_df.head()

,CHROM,REF,ALT,sample_1_0,sample_1_1,sample_1_2,sample_1_3,sample_1_4,sample_1_5,sample_1_6,...,sample_1_996,sample_1_997,sample_1_998,sample_1_999,alt1,alt2,alt3,alt4,alt5,alt6
POS,,,,,,,,,,,,,,,,,,,,,
47219343,1,A,G,"[A, A]","[A, A]","[A, A]","[A, A]","[A, A]","[A, A]","[A, A]",...,"[A, A]","[A, A]","[A, A]","[A, A]",G,None,None,None,None,None
47219348,1,T,C,"[T, T]","[T, T]","[T, T]","[T, T]","[T, T]","[T, T]","[T, T]",...,"[T, T]","[T, T]","[T, T]","[T, T]",C,None,None,None,None,None
47219350,1,G,A,"[G, G]","[G, G]","[G, G]","[G, G]","[G, G]","[G, G]","[G, G]",...,"[G, G]","[G, G]","[G, G]","[G, G]",A,None,None,None,None,None
47219370,1,A,T,"[A, A]","[A, A]","[A, A]","[A, A]","[A, A]","[A, A]","[A, A]",...,"[A, A]","[A, A]","[A, A]","[A, A]",T,None,None,None,None,None
47219382,1,C,A,"[C, C]","[C, C]","[C, C]","[C, C]","[C, C]","[C, C]","[C, C]",...,"[C, C]","[C, C]","[C, C]","[C, C]",A,None,None,None,None,None


In [22]:
# begin creating final DF
df = seq_int_df[[s for s in seq_int_df.columns if s.startswith('s')]].T
#sequence df
df.head()

POS,47219343,47219348,47219350,47219370,47219382,47219400,47219406,47219415,47219418,47219424,...,47225921,47225922,47225924,47225925,47225929,47225933,47225934,47225936,47225939,47225940
sample_1_0,"[A, A]","[T, T]","[G, G]","[A, A]","[C, C]","[C, C]","[C, C]","[A, A]","[A, A]","[G, G]",...,"[C, C]","[G, G]","[T, T]","[C, C]","[C, C]","[C, C]","[G, G]","[C, C]","[T, T]","[G, G]"
sample_1_1,"[A, A]","[T, T]","[G, G]","[A, A]","[C, C]","[C, C]","[C, C]","[A, A]","[A, A]","[G, G]",...,"[C, C]","[G, G]","[T, T]","[C, C]","[C, C]","[C, C]","[G, G]","[C, C]","[T, T]","[G, G]"
sample_1_2,"[A, A]","[T, T]","[G, G]","[A, A]","[C, C]","[C, C]","[C, C]","[A, A]","[A, A]","[G, G]",...,"[C, C]","[G, G]","[T, T]","[C, C]","[C, C]","[C, C]","[G, G]","[C, C]","[T, T]","[G, G]"
sample_1_3,"[A, A]","[T, T]","[G, G]","[A, A]","[C, C]","[C, C]","[C, C]","[A, A]","[A, A]","[G, G]",...,"[C, C]","[G, G]","[T, T]","[C, C]","[C, C]","[C, C]","[G, G]","[C, C]","[T, T]","[G, G]"
sample_1_4,"[A, A]","[T, T]","[G, G]","[A, A]","[C, C]","[C, C]","[C, C]","[A, A]","[A, A]","[G, G]",...,"[C, C]","[G, G]","[T, T]","[C, C]","[C, C]","[C, C]","[G, G]","[C, C]","[T, T]","[G, G]"


In [23]:
# labels df
joined_df = joined_df.set_index('sample')
joined_df.index.name = None

In [24]:
df = joined_df.join(df)

In [25]:
data_dir = '/Users/djemec/data/'
file = 'var_effect/test_1k_tal1_missing.csv'
df.index.name = 'sample'
df.to_csv(data_dir+file, index=True)